In [1]:
# Import Libraries
import tweepy as tw
import configparser
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import re
import nltk
import datetime
from datetime import datetime as datetime_price
import pytz
from dateutil.relativedelta import relativedelta
import time

# Authentication for API

In [3]:
keys = pd.read_csv(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\Input\TW API Keys\API_2\apikeys_2.csv")

my_api_key = keys["Keys"][0]
my_api_secret = keys["Keys"][1]

my_access_token = keys["Keys"][2]
my_access_token_secret = keys["Keys"][3]

In [4]:
# your Twitter API key ans access token
api_key = my_api_key
api_secret = my_api_secret

access_token = my_access_token
access_token_secret = my_access_token_secret

In [5]:
# authenticate
auth = tw.OAuthHandler(my_api_key, my_api_secret)
auth.set_access_token(access_token, access_token_secret)

In [6]:
api = tw.API(auth, wait_on_rate_limit=True)

# Insert the List of Users

In [7]:
user_df = pd.read_csv(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\Input\Top 20 Cryptocurrencies and Related TW Accounts.csv", encoding= 'unicode_escape')
user_df

,Coin,ID,Owner/Influence,Name,Screen_name,Status,Why did I choose this person?
0,Ethereum,ethereum,Vitalik Buterin,vitalik.eth,VitalikButerin,co-founder,he is also known in crypto world
1,BNB,binancecoin,Changpeng Zhao,CZ ??Binance,cz_binance,CEO,NaN
2,XRP,ripple,Brad Garlinghouse,Brad Garlinghouse,bgarlinghouse,CEO,NaN
3,ADA(Cardano),cardano,Charles Hoskinson,Charles Hoskinson,IOHK_Charles,CEO/co-founder,NaN
4,DOGE,dogecoin,Elon Musk_doge,Elon Musk,elonmusk,influencer,"as a popular figure, highly interested in this..."
5,MATIC(Polygon),matic-network,Sandeep Nailwal,Sandeep | Polygon ????3??,sandeepnailwal,co-founder,he seems more active than other co-founders
6,DOT(Polkadot),polkadot,WEB3 Foundation,Web3 Foundation,Web3foundation,foundation,there is no account of CEO or sth
7,HEX,hex,Richard Heart,Richard Heart,RichardHeartWin,CEO,NaN
8,SHIB,shiba-inu,Elon Musk_shiba,Elon Musk,elonmusk,influencer,"as a popular figure, highly interested in this..."
9,SOL,solana,Solana,Solana,solana,company,there is no account of CEO or sth


# Taking Tweets of List of Users

user = "el33th4xor"
limit = 300

In [8]:
#datetime format -> (year, month, day, hour, minute, second)
endDate = datetime.datetime.now()
startDate = endDate - relativedelta(years = 7)
#endDate = datetime.datetime.now()
#startDate = endDate - relativedelta(days = 1) #days, months, years can be used to calculate

utc=pytz.UTC

startDate = utc.localize(startDate) 
endDate = utc.localize(endDate)

In [9]:
startDate

datetime.datetime(2016, 2, 12, 19, 13, 10, 500062, tzinfo=<UTC>)

In [10]:
columns = ["Tweet_id", "Name", "Screen_name", "Tweet", "Date", "Category", "N_of_RT", "N_of_fav", "Related_coin"]

--old version of taking tweets
#cursor can be used for just one time, be careful about that, save and use use data frame/s for next steps
tweets = tw.Cursor(api.user_timeline, 
                   screen_name = user, count = 200, 
                   tweet_mode = "extended").items(limit)

In [11]:
def take_tweets(sc_name):
    tweets = []
    tmpTweets = api.user_timeline(screen_name = sc_name)
    if(len(tmpTweets) == 0):
        print("First request for/", sc_name, "/is unseccessful")
    for tweet in tmpTweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            tweets.append(tweet)
    
    while tmpTweets[-1].created_at > startDate:
        old_ID = tmpTweets[-1].id
        tmpTweets = api.user_timeline(screen_name = sc_name, max_id = old_ID)
        while(len(tmpTweets) == 0):
            #print("empty list, please try again")
            tmpTweets = api.user_timeline(screen_name = sc_name, max_id = old_ID)
        #print("1: ", {tmpTweets[0].created_at})
        #print("2: ", {tmpTweets[-1].created_at})
        for tweet in tmpTweets:
            if tweet.created_at < endDate and tweet.created_at > startDate:
                tweets.append(tweet)
    return tweets

In [12]:
def take_tweets_2(sc_name):
    #variables
    start = time.time()
    tweets = []
    num_of_tweets = 0
    
    #initial request
    while True:
        try:
            tmpTweets = api.user_timeline(screen_name = sc_name)
            while(len(tmpTweets) == 0):
                print("empty list error_1, please try again")
                tmpTweets = api.user_timeline(screen_name = sc_name)
            for tweet in tmpTweets:
                if tweet.created_at < endDate and tweet.created_at > startDate:
                    tweets.append(tweet)
                    num_of_tweets += 1
            break
        except tw.errors.TweepyException as e:
            print("Going to sleep")
            print(e)
            time.sleep(60*15)
            continue
    
    #collection
    while True:
        try:
            while (tmpTweets[-1].created_at > startDate) and (num_of_tweets < 3400) and (len(tmpTweets) != 1):
                old_ID = tmpTweets[-1].id
                tmpTweets = api.user_timeline(screen_name = sc_name, max_id = old_ID)
                while(len(tmpTweets) == 0):
                    print("empty list error_2, please try again")
                    print(old_ID)
                    time.sleep(15)
                    tmpTweets = api.user_timeline(screen_name = sc_name, max_id = old_ID)
                    #print("1: ", {tmpTweets[0].created_at})
                for tweet in tmpTweets:
                    if tweet.created_at < endDate and tweet.created_at > startDate:
                        tweets.append(tweet)
                        num_of_tweets += 1
            break
        except tw.errors.TweepyException as e:
            print(e)
            print("Going to sleep")
            time.sleep(60*15)
            continue
    
    #return        
    end = time.time()
    print(sc_name)
    print(end-start)
    return tweets

In [13]:
def take_tweets_ID(sc_name, max_ID):
    tweets = []
     
    while True:
        try:
            tmpTweets = api.user_timeline(screen_name = sc_name, max_id = max_ID)
            while(len(tmpTweets) == 0):
                #print("empty list, please try again")
                tmpTweets = api.user_timeline(screen_name = sc_name, max_id = max_ID)
            for tweet in tmpTweets:
                if tweet.created_at < endDate and tweet.created_at > startDate:
                    tweets.append(tweet)
            break
        except tw.errors.TweepyException as e:
            print("Going to sleep")
            print(e)
            time.sleep(60*15)
            continue
    
    while True:
        try:
            while tmpTweets[-1].created_at > startDate:
                old_ID = tmpTweets[-1].id
                tmpTweets = api.user_timeline(screen_name = sc_name, max_id = old_ID)
                while(len(tmpTweets) == 0):
                    #print("empty list, please try again")
                    tmpTweets = api.user_timeline(screen_name = sc_name, max_id = old_ID)
                    #print("1: ", {tmpTweets[0].created_at})
                for tweet in tmpTweets:
                    if tweet.created_at < endDate and tweet.created_at > startDate:
                        tweets.append(tweet)
            break
        except tw.errors.TweepyException as e:
            print(e)
            time.sleep(60*15)
            continue
    print(sc_name)    
    return tweets

In [14]:
def categorize_tweets(tweets):
    
    data = []
    term_rt = "RT"
    for tweet in tweets:
    
        #for consecutive tweets, next one is shown primaryly in data frame, I can check with using hours and minutes if necessary
        date = tweet.created_at
        #date = date.strftime('%Y-%m-%d')
    
        #retweet and favorite counts
        number_of_retweet = tweet.retweet_count
        number_of_favorite = tweet.favorite_count
    
    
        #it's gonna be direct tweet if all the checks below fail
        category_specified = "direct"

        #retweet check
        if term_rt in tweet.text.split():

            category_specified = "retweet"

        #quote(rt with comment) check
        elif tweet.is_quote_status is True:
            category_specified = "quote"

        #reply check
        elif tweet.in_reply_to_status_id is not None:
            category_specified = "reply"

        data.append([tweet.id, tweet.user.name, tweet.user.screen_name, tweet.text, date, category_specified, number_of_retweet, number_of_favorite, ""])
    
    df_met = pd.DataFrame(data, columns = columns)
    return df_met

In [15]:
def tweetCollector(user_df):
    df = pd.DataFrame(columns = columns)

    index_counter_1 = 0;
    for x in user_df.Screen_name:
    
        taken_tweets = take_tweets_2(x)
        df_met_2 = categorize_tweets(taken_tweets)

        df_met_2["Related_coin"] = user_df["ID"][index_counter_1]
        index_counter_1 += 1

        df = df.append(df_met_2)

    df = df[["Tweet_id", "Name", "Screen_name", "Related_coin", "Tweet", "Date", "Category", "N_of_RT", "N_of_fav"]]
    
    return df

In [16]:
def oneCollector(sc_name):
    df = pd.DataFrame(columns = columns)
    
    taken_tweets = take_tweets_2(sc_name)
    df_met_2 = categorize_tweets(taken_tweets)

    df_met_2["Related_coin"] = sc_name

    df = df.append(df_met_2)
    df = df[["Tweet_id", "Name", "Screen_name", "Related_coin", "Tweet", "Date", "Category", "N_of_RT", "N_of_fav"]]
    
    return df

In [17]:
def oneCollector_ID(sc_name, max_ID):
    df = pd.DataFrame(columns = columns)
    
    taken_tweets = take_tweets_ID(sc_name, max_ID)
    df_met_2 = categorize_tweets(taken_tweets)

    df_met_2["Related_coin"] = sc_name

    df = df.append(df_met_2)
    df = df[["Tweet_id", "Name", "Screen_name", "Related_coin", "Tweet", "Date", "Category", "N_of_RT", "N_of_fav"]]
    
    return df

In [18]:
#oneCollector example
#sc_n = "cz_binance"
#df_one = oneCollector(sc_n)
#df_one

In [19]:
#tweetCollector example
#right now df is used by tweetCollector
start = time.time()
df = tweetCollector(user_df)
end = time.time()
print(end - start)

empty list error_2, please try again
1599059086141595648
empty list error_2, please try again
1527076952531644417
VitalikButerin
100.6792950630188
empty list error_2, please try again
1609448577767542784
empty list error_2, please try again
1603106233472385028
empty list error_2, please try again
1596056452090986496
empty list error_2, please try again
1592870560039866369
empty list error_2, please try again
1590375374353215496
empty list error_2, please try again
1582714641154682880
empty list error_2, please try again
1572184722255785984
empty list error_2, please try again
1564856740918722560
empty list error_2, please try again
1564856740918722560
empty list error_2, please try again
1556803901596262402
cz_binance
225.02441501617432
bgarlinghouse
20.509400844573975
IOHK_Charles
76.06998777389526
elonmusk
66.62741088867188
empty list error_2, please try again
1587338901727113216
empty list error_2, please try again
1569039289035407364


Rate limit reached. Sleeping for: 304


Failed to send request: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Going to sleep
sandeepnailwal
1319.819546699524
Web3foundation
59.07002305984497
RichardHeartWin
68.77224349975586
elonmusk
67.36814570426941
empty list error_2, please try again
1483551474009481217
empty list error_2, please try again
1466430455763259392
empty list error_2, please try again
1433895206412292107
solana
127.73270273208618
justinsuntron
81.25381755828857
empty list error_2, please try again
1471634948209745927


Rate limit reached. Sleeping for: 448


Uniswap
503.149861574173
SatoshiLite
76.32275462150574
empty list error_2, please try again
1615266540064243714
empty list error_2, please try again
1472706102311723015
el33th4xor
109.85149693489075
SergeyNazarov
7.820849180221558
buchmanster
73.31823825836182
2984.1802520751953


In [20]:
"""#right now df is used by oneCollector
sc_n = "justinsuntron"
df = oneCollector(sc_n)
df"""

'#right now df is used by oneCollector\nsc_n = "justinsuntron"\ndf = oneCollector(sc_n)\ndf'

In [21]:
df

,Tweet_id,Name,Screen_name,Related_coin,Tweet,Date,Category,N_of_RT,N_of_fav
0,1618483820717834240,vitalik.eth,VitalikButerin,ethereum,RT @reflexerfinance: Check out @VitalikButerin...,2023-01-26 05:39:47+00:00,retweet,177,0
1,1616527030790623233,vitalik.eth,VitalikButerin,ethereum,An incomplete guide to stealth addresses:\n\nh...,2023-01-20 20:04:12+00:00,direct,1098,4034
2,1609386762333478912,vitalik.eth,VitalikButerin,ethereum,"RT @GovofCO: This time of year, it is essentia...",2023-01-01 03:11:19+00:00,retweet,381,0
3,1609345489790730240,vitalik.eth,VitalikButerin,ethereum,RT @JBSchweitzer: As the year of the Merge end...,2023-01-01 00:27:19+00:00,retweet,324,0
4,1609339278445907968,vitalik.eth,VitalikButerin,ethereum,@mukuroeth @ChenWainuo @zengjiajun_eth @BlockG...,2023-01-01 00:02:38+00:00,reply,27,165
...,...,...,...,...,...,...,...,...,...
3412,1472791649826881538,"Ethan Buchman (🐝,🦇)",buchmanster,cosmos,@MpeterF @Econgrad5143 @DEhnts @HenricCont @vi...,2021-12-20 04:51:07+00:00,reply,0,0
3413,1472714294643109892,"Ethan Buchman (🐝,🦇)",buchmanster,cosmos,@MpeterF @Econgrad5143 @DEhnts @HenricCont @vi...,2021-12-19 23:43:44+00:00,reply,0,0
3414,1472713854585126918,"Ethan Buchman (🐝,🦇)",buchmanster,cosmos,@TheStalwart @leee_harris I guess generally it...,2021-12-19 23:41:59+00:00,quote,0,0
3415,1472711376909000704,"Ethan Buchman (🐝,🦇)",buchmanster,cosmos,@MpeterF @Econgrad5143 @DEhnts @HenricCont @vi...,2021-12-19 23:32:08+00:00,reply,0,1


In [22]:
#removing duplications, reseting index
df.drop_duplicates(inplace = True)
df.reset_index(inplace = True, drop = True)

In [23]:
#seperating dat-month-year
df['Day'] = pd.DatetimeIndex(df['Date']).day
df['Month'] = pd.DatetimeIndex(df['Date']).month
df['Year'] = pd.DatetimeIndex(df['Date']).year

In [24]:
df.shape

(44092, 12)

In [25]:
#null check --> if it is "False", we all ok
df.isna().any().any()

False

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44092 entries, 0 to 44091
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Tweet_id      44092 non-null  object
 1   Name          44092 non-null  object
 2   Screen_name   44092 non-null  object
 3   Related_coin  44092 non-null  object
 4   Tweet         44092 non-null  object
 5   Date          44092 non-null  object
 6   Category      44092 non-null  object
 7   N_of_RT       44092 non-null  object
 8   N_of_fav      44092 non-null  object
 9   Day           44092 non-null  int64 
 10  Month         44092 non-null  int64 
 11  Year          44092 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 4.0+ MB


In [27]:
#duplication check
df[df.duplicated(keep=False)].shape[0]

0

In [28]:
df['Date'] = df['Date'].dt.tz_localize(None)

In [29]:
#check .xlsx file name
pd.set_option("max_rows", None)
df.to_excel(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\RA_Cryptocurrency\Data\Tweet Data\tweets_all_7years.xlsx")
df.head()

,Tweet_id,Name,Screen_name,Related_coin,Tweet,Date,Category,N_of_RT,N_of_fav,Day,Month,Year
0,1618483820717834240,vitalik.eth,VitalikButerin,ethereum,RT @reflexerfinance: Check out @VitalikButerin...,2023-01-26 05:39:47,retweet,177,0,26,1,2023
1,1616527030790623233,vitalik.eth,VitalikButerin,ethereum,An incomplete guide to stealth addresses:\n\nh...,2023-01-20 20:04:12,direct,1098,4034,20,1,2023
2,1609386762333478912,vitalik.eth,VitalikButerin,ethereum,"RT @GovofCO: This time of year, it is essentia...",2023-01-01 03:11:19,retweet,381,0,1,1,2023
3,1609345489790730240,vitalik.eth,VitalikButerin,ethereum,RT @JBSchweitzer: As the year of the Merge end...,2023-01-01 00:27:19,retweet,324,0,1,1,2023
4,1609339278445907968,vitalik.eth,VitalikButerin,ethereum,@mukuroeth @ChenWainuo @zengjiajun_eth @BlockG...,2023-01-01 00:02:38,reply,27,165,1,1,2023


In [ ]:
#seperation according to categories 
retweet_df = df[df["Category"] == "retweet"]
retweet_df.reset_index(inplace = True, drop=True)
quote_df = df[df["Category"] == "quote"]
quote_df.reset_index(inplace = True, drop=True)
reply_df = df[df["Category"] == "reply"]
reply_df.reset_index(inplace = True, drop=True)
direct_df = df[df["Category"] == "direct"]
direct_df.reset_index(inplace = True, drop=True)
retweet_df.head()

# Taking User Object

In [ ]:
user_try = api.get_user(screen_name = "el33th4xor")
user_try.name

# Follower - Followed

check this part of code later??????????????? cursor object is not itterable?

In [ ]:
user = "el33th4xor"
followers = []
get_followers = api.get_followers(screen_name = user)
#get_followers = tw.Cursor(api.get_followers, id = user)

for user in get_followers:
    followers.append(user)

In [ ]:
len(followers)

In [ ]:
columns_follower = ["User_id", "Name", "Screen_name", "Verification_st", "Followers_count", "Followed_count"]
data_followers = []

In [ ]:
for user in followers:
    data_followers.append([user.id, user.name, user.screen_name, user.verified, user.followers_count, user.friends_count])

In [ ]:
df_follower = pd.DataFrame(data_followers, columns = columns_follower)
df_follower

In [ ]:
df_follower.drop_duplicates(inplace = True)
df_follower.reset_index(inplace = True, drop = True)

# Statistics and Visualization

Graph should be addepted to new version of df(with lots of users)

In [ ]:
df.corr()

In [ ]:
plot1 = retweet_df.groupby("Day")["Tweet_id"].count()
plot2 = quote_df.groupby("Day")["Tweet_id"].count()
plot3 = reply_df.groupby("Day")["Tweet_id"].count()
plot4 = direct_df.groupby("Day")["Tweet_id"].count()
plt.figure(figsize = (14,7))
ax= plt.subplot()
ax.bar(plot1.index-0.3, plot1.values, width = 0.2, color = "pink", align = "center", label = "Retweet")
ax.bar(plot2.index-0.1, plot2.values, width = 0.2, color = "yellow", align = "center", label = "Quote")
ax.bar(plot3.index+0.1, plot3.values, width = 0.2, color = "purple", align = "center", label = "Reply")
ax.bar(plot4.index+0.3, plot4.values, width = 0.2, color = "grey", align = "center", label = "Direct")
plt.title("Number of Tweets According to Category per Day")
plt.ylabel("Number of Tweets")
plt.xlabel("Days")
#they are not sorted from 1 to 30 or 31, last they is today and first day is one month earlier. There is need for a solution also.
plt.xticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]) #find a solution for changes of number of days in a month
plt.legend()
plt.show()